In [2]:
import pandas as pd

melb_df = pd.read_csv('data/melb_data_ps.csv', sep = ',')

In [3]:
melb_df['Address'].nunique()

13378

In [5]:
## Извлечём из признака адреса характеристику подтипа улицы (улица, шоссе, авеню, бульвар)
# На вход данной функции поступает строка с адресом.
def get_street_type(address):
    # Создаём список географических пометок exclude_list.
    exclude_list = ['N', 'S', 'W', 'E']
    # Метод split() разбивает строку на слова по пробелу.
    # В результате получаем список слов в строке и заносим его в переменную address_list.
    address_list = address.split(' ')
    # Обрезаем список, оставляя в нём только последний элемент,
    # потенциальный подтип улицы, и заносим в переменную street_type. 
    street_type = address_list[-1]
    # Делаем проверку на то, что полученный подтип является географической пометкой.
    # Для этого проверяем его на наличие в списке exclude_list. 
    if street_type in exclude_list:
    # Если переменная street_type является географической пометкой,
    # переопределяем её на второй элемент с конца списка address_list.
        return address_list[-2]
    return street_type

In [76]:
street_types = melb_df['Address'].apply(get_street_type)
melb_df['StreetTypes'] = street_types
melb_df['StreetTypes'].nunique()

56

In [81]:
def eliminate_longs(address):
    small_list = []
    for a in melb_df['StreetTypes'].unique():
        if len(a) <= 3:
            small_list.append(a)
    for b in small_list:
        if set(b).issubset(set(address)):
            return b
    return address

melb_df['StreetTypes'] = melb_df['StreetTypes'].apply(eliminate_longs)

In [85]:
display(melb_df['StreetTypes'].nunique())
display(melb_df['StreetTypes'].value_counts())

36

St          8020
Rd          2827
Ct           648
Dr           447
Av           362
Gr           323
Pde          226
Pl           169
Cr           156
Cl           100
La            67
Bvd           66
Tce           48
Wy            40
Hwy           25
Sq            11
Esp            8
Cir            7
Gdns           4
Fairway        4
Mews           4
Righi          3
Victoria       2
Glade          1
Nook           1
Qy             1
Cove           1
Res            1
Loop           1
East           1
Dell           1
Eyrie          1
Athol          1
Hts            1
Outlook        1
Woodland       1
Name: StreetTypes, dtype: int64

In [86]:
popular_stypes = street_types.value_counts().nlargest(10).index
display(popular_stypes)

Index(['St', 'Rd', 'Ct', 'Dr', 'Av', 'Gr', 'Pde', 'Pl', 'Cr', 'Cl'], dtype='object')

In [87]:
melb_df['StreetType'] = street_types.apply(lambda x: x if x in popular_stypes else 'other')
melb_df['StreetType'].nunique()

11

In [88]:
melb_df.drop('Address', axis=1, inplace = True)